In [5]:
# Imports
import copy
import numpy as np
import random

In [25]:
# General usage

def precision(TP, FP):
    return TP/(TP+FP)

def recall(TP, FN):
    return TP/(TP+FN)

def f1_score(TP, FP, FN):
    return TP/(TP+0.5*(FP+FN))

def associator(gt_bboxes, pred_bboxes, conf, type:str):
    # Reorganize by confidence value
    conf, pred_bboxes = zip(*reversed(sorted(zip(conf, pred_bboxes))))
    conf = list(conf)
    pred_bboxes = list(pred_bboxes)
    assert len(pred_bboxes) == len(conf), f"The quantity of boxes doesn't match the number of confidence values"
    iou_table = []
    # Rows --> Ground Truth
    for gt in gt_bboxes:
        line = []
        # Columns --> Predictions
        for pred in pred_bboxes:
            iou = IoU_calc(gt ,pred, type)
            line.append(iou)
        iou_table.append(line)
    iou_table = np.asarray(iou_table)
    maxim = np.max(iou_table)
    associated = []
    while maxim > 0 and iou_table.shape[1]>0:
        row_idx, column_idx = np.unravel_index(np.argmax(iou_table), iou_table.shape)
        # Caso o máximo seja único na linha e na coluna
        if np.count_nonzero(iou_table[row_idx, :] == maxim) == 1 and np.count_nonzero(iou_table[:, column_idx] == maxim) == 1:
            associated.append([gt_bboxes[row_idx], pred_bboxes[column_idx], iou_table[row_idx, column_idx]])
            iou_table = np.delete(iou_table, obj=row_idx, axis=0) # axis=0 --> linhas
            iou_table = np.delete(iou_table, obj=column_idx, axis=1)
            del gt_bboxes[row_idx]
            del pred_bboxes[column_idx]
            del conf[column_idx]
        else:
            # Obter posições com valor máximo na mesma linha e coluna
            max_idx = np.argwhere(iou_table == maxim)
            idx_remove = []
            for idx, [row, column] in enumerate(max_idx):
                if (row != row_idx and column != column_idx):
                    idx_remove.append(idx)
            max_idx = np.delete(max_idx, obj=idx_remove, axis=0)
            # Remover os que têm menor conf
            confs = [conf[confi] for confi in max_idx[:,1]]
            confi = 0
            idx_keep = []
            first_run = 1
            for idx, confd in enumerate(confs):
                if confd > confi:
                    confi = confd
                    idx_keep = [idx]
                elif confd == confi:
                    idx_keep.append(idx)
            max_idx = max_idx[idx_keep]
            # Remover de forma aleatória
            if max_idx.shape[0] > 1:
                max_idx = max_idx[random.randint(0,max_idx.shape[0]-1)]
            max_idx = np.squeeze(max_idx)
            associated.append([gt_bboxes[max_idx[0]], pred_bboxes[max_idx[1]], iou_table[max_idx[0], max_idx[1]]])
            iou_table = np.delete(iou_table, obj=max_idx[0], axis=0)
            iou_table = np.delete(iou_table, obj=max_idx[1], axis=1)
            del gt_bboxes[max_idx[0]]
            del pred_bboxes[max_idx[1]]
            del conf[max_idx[1]]
        if iou_table.shape[1]<=0:
            break
        maxim = np.max(iou_table)
    for box in gt_bboxes:
        associated.append([box, None, 0])
    for box in pred_bboxes:
        associated.append([None, box, 0])
    return associated

In [1]:
# IoU calculations

def IoU_mask(maskA, maskB):
    intersection = np.logical_and(maskA, maskB)
    union = np.logical_or(maskA, maskB)
    iou = np.sum(intersection) / np.sum(union)
    return iou
    
def IoU_bbox(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute IoU
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou
    
def IoU_calc(gt ,pred, type:str):
    if type == "box":
        iou = IoU_bbox(gt, pred)
        return iou
    elif type == "mask":
        iou = IoU_mask(gt, pred)
        return iou
    else:
        raise Exception("Invalid IoU type")
    

In [31]:
# For object detection and instance segmentation

def AP_aux(iou, iou_tresh):
    # return TP, FP, FN
    if iou >= iou_tresh: # False positive
        return 1, 0, 0
    elif iou < iou_tresh and iou > 0: # False positive and false negative
        return 0, 1, 1
    else:
        raise Exception("Invalid IoU")
        
def Evaluation_TFPN(associated_boxes, iou_tresh):
    TP, FP, FN = 0, 0, 0
    for element in associated_boxes:
        gt_bbox, pred_bbox, iou = element
        if gt_bbox is None:
            FP += 1
        elif pred_bbox is None:
            FN += 1
        else:
            TPn, FPn, FNn = AP_aux(iou, iou_tresh)
            TP, FP, FN = TP + TPn, FP + FPn, FN + FNn
    return TP, FP, FN

def precision_recall(gt_bboxes, pred_bboxes, conf, iou_tresh, type):
    gt_bboxes_send = copy.deepcopy(gt_bboxes)
    pred_bboxes_send = copy.deepcopy(pred_bboxes)
    conf_send = copy.deepcopy(conf)
    # TP, FP, FN = Evaluation_TFPN(box_association(gt_bboxes_send, pred_bboxes_send, conf_send),iou_tresh)
    TP, FP, FN = Evaluation_TFPN(associator(gt_bboxes_send, pred_bboxes_send, conf_send, type),iou_tresh)
    return precision(TP, FP), recall(TP, FN)

        
    

In [32]:
# Object detection test
gt_bboxes = [(0,0,1,1), (0,0,2,1), (0,0,3,1), (1000, 1000, 2000, 2000), (0,0,3,40), (0,0,3,45)]
pred_bboxes = [(0,0,2,1), (0,0,2,1), (0,0,3,1), (3000, 3000, 4000, 4000),(0,0,3,1)]
conf = [0.5, 0.5, 0.8, 1, 0.5]
    
# A = box_association(gt_bboxes, pred_bboxes, conf)
# TP, FP, FN = Evaluation_TFPN(A, 0.7)
# print(precision(TP, FP))
# recall(TP, FN)
print(precision_recall(gt_bboxes, pred_bboxes, conf, 0, "box"))


(0.8, 0.6666666666666666)